In [1]:
! uv pip install langchain tiktoken groq rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.12.7 environment at: /home/deblina/Documents/multi-doc-chatbot/.venv
Audited 6 packages in 49ms


In [51]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import os
from dotenv import load_dotenv

In [3]:
loader = TextLoader("/home/deblina/Documents/multi-doc-chatbot/data/agenticAI.txt")

In [4]:
documents = loader.load()

In [ ]:


text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)

In [6]:
text_chunks = text_splitter.split_documents(documents=documents)

In [7]:
text_chunks[0].page_content[:200]

'Understanding Agentic AI'

In [8]:
! pip install -U langchain-huggingface

In [10]:
!uv pip install fastembed

Using Python 3.12.7 environment at: /home/deblina/Documents/multi-doc-chatbot/.venv
Audited 1 package in 11ms


In [12]:
import tqdm as notebook_tqdm

In [13]:
embedding = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [14]:
!uv pip install faiss-cpu

Using Python 3.12.7 environment at: /home/deblina/Documents/multi-doc-chatbot/.venv
Audited 1 package in 4ms


In [16]:
vec_store = FAISS.from_documents(embedding=embedding, documents=text_chunks)

In [17]:
retriever = vec_store.as_retriever()

In [18]:
query = "future aspect of agentic ai"

docs = vec_store.similarity_search(query=query, k=4)

In [19]:
docs

[Document(id='c9390732-f444-49d6-817f-a88c23d445aa', metadata={'source': '/home/deblina/Documents/multi-doc-chatbot/data/agenticAI.txt'}, page_content='The Future of Agentic AI'),
 Document(id='7bd6ad7f-0774-45c7-8469-884cb7987b8e', metadata={'source': '/home/deblina/Documents/multi-doc-chatbot/data/agenticAI.txt'}, page_content='Applications of Agentic AI'),
 Document(id='79426b2d-8e9d-4e99-a343-59e9ee9f6e41', metadata={'source': '/home/deblina/Documents/multi-doc-chatbot/data/agenticAI.txt'}, page_content='Understanding Agentic AI'),
 Document(id='c28f7942-aa03-4455-852e-4f6b999b534e', metadata={'source': '/home/deblina/Documents/multi-doc-chatbot/data/agenticAI.txt'}, page_content='As agentic AI continues to evolve, it is expected to bring about significant transformations in how we interact with technology and how complex problems are solved. Challenges such as ensuring safety, interpretability, and ethical decision-making remain critical areas of research and development')]

In [21]:
for i,doc in enumerate(docs):
    print(f"documents{i+1}")
    print(doc)

documents1
page_content='The Future of Agentic AI' metadata={'source': '/home/deblina/Documents/multi-doc-chatbot/data/agenticAI.txt'}
documents2
page_content='Applications of Agentic AI' metadata={'source': '/home/deblina/Documents/multi-doc-chatbot/data/agenticAI.txt'}
documents3
page_content='Understanding Agentic AI' metadata={'source': '/home/deblina/Documents/multi-doc-chatbot/data/agenticAI.txt'}
documents4
page_content='As agentic AI continues to evolve, it is expected to bring about significant transformations in how we interact with technology and how complex problems are solved. Challenges such as ensuring safety, interpretability, and ethical decision-making remain critical areas of research and development' metadata={'source': '/home/deblina/Documents/multi-doc-chatbot/data/agenticAI.txt'}


In [23]:
!uv pip install -U langchain-groq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.12.7 environment at: /home/deblina/Documents/multi-doc-chatbot/.venv
Resolved 29 packages in 1.03s                                        
⠙ Preparing packages... (0/3)                                                   
⠙ Preparing packages... (0/3)--------------     0 B/134.27 KiB          
⠙ Preparing packages... (0/3)-------------- 16.00 KiB/134.27 KiB        
⠙ Preparing packages... (0/3)-------------- 32.00 KiB/134.27 KiB        
⠙ Preparing packages... (0/3)-------------- 48.00 KiB/134.27 KiB        
⠙ Preparing packages... (0/3)-------------- 48.00 KiB/134.27 KiB        
groq                 ------------------------------ 48.00 KiB/134.27 KiB
⠙ Preparing packages... (0/3)--------------     0 B/473.49 KiB          
groq                 ------------------------------ 48.00 KiB/134.27 KiB
⠙ Preparing packages... (0/3)-------------- 14.88 KiB/473.49 KiB        
groq                 ------------------------------ 64.00 KiB/134.27 KiB
⠙ Preparing packages... (0/3)------

In [59]:
load_dotenv()
api_key = os.getenv("groq_api")

In [60]:
model = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=api_key
)

In [61]:
template = """
you are an assistant for question answering tasks.
use the following pieces of retrieved context to answer the question.
if you dont know the answer, just say you dont know.
use ten sentences maximum and keep the answer concise.
Question : {question}
context : {context}
Answer :
"""

In [62]:
prompt = ChatPromptTemplate.from_template(template)

In [63]:
rag_chain = (
    {'context': retriever, 'question': RunnablePassthrough()} 
    | prompt
    | model
    | StrOutputParser()
)

In [64]:
answer = rag_chain.invoke({"question":"future aspect of ai agents"})

In [65]:
answer

'The future of agentic AI agents is expected to be highly adaptable and learning-based, allowing them to adjust their behavior and improve their performance based on new information and experiences. They will be able to learn from their mistakes and apply that knowledge to future situations. This adaptability will enable them to tackle complex problems and make decisions with increased accuracy. Additionally, agentic AI agents will be able to improve their performance over time, making them increasingly effective in various tasks and applications. Their ability to learn and adapt will also enable them to work effectively in dynamic and unpredictable environments. The future of agentic AI agents holds great promise for revolutionizing various industries and aspects of life. However, the development and implementation of agentic AI will require careful consideration and oversight to ensure they align with human values and ethics. The future of agentic AI is expected to be highly transfor